__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-26 11:14:26 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-26 11:14:26 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-26 11:14:29 | AdsListParser: downloader init
[INFO    ] 2022-09-26 11:14:29 | AvitoParser: init
[INFO    ] 2022-09-26 11:14:37 | AvitoParser: 76 pages for read
[INFO    ] 2022-09-26 11:14:37 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-26 11:14:41 | AdsListParser: read page 1
[INFO    ] 2022-09-26 11:14:47 | AdsListParser: read page 2
[INFO    ] 2022-09-26 11:14:52 | AdsListParser: read page 3
[INFO    ] 2022-09-26 11:14:57 | AdsListParser: read page 4
[INFO    ] 2022-09-26 11:15:02 | AdsListParser: read page 5
[INFO    ] 2022-09-26 11:15:08 | AdsListParser: read page 6
[INFO    ] 2022-09-26 11:15:15 | AdsListParser: read page 7
[INFO    ] 2022-09-26 11:15:21 | AdsListParser: read page 8
[INFO    ] 2022-09-26 11:15:25 | AdsListParser: read page 9
[INFO    ] 2022-09-26 11:15:32 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3796
3576


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
222,2534001900,"Объявление «1-к. квартира, 48 м², 8/10 эт.» 13 фотографий",6500000,,"ул. Комбрига Потапова, 31",! Супер Цена! ЖК Архитектор.\n\nПродаю солнечную уютную 1 комнатную квартиру с шикарным видом на море из всех окон! + большая утеплённая лоджия (использовали её как полноценную жилую комнату). В квартире остаётся вся мебель и техника для комфортного проживан,5,2022-09-26 11:20:17.001605,sevastopol
3490,2456838563,"Объявление «2-к. квартира, 81 м², 3/8 эт.» 21 фотография",12900000,,"ул. Тараса Шевченко, 49к2","Двухкомнатная квартира 81 м2, на 3/8 этаже дома с просторной кухней 14 м2. Расположена в г. Севастополь, Гагаринский район, ул. Тараса Шевченко, д. 49/2. \n\nПозвоните нам, чтобы узнать все детали! \n\nТехнические характеристики: \n— Адрес: г. Севастополь, Гага",75,2022-09-26 11:20:17.001605,sevastopol


In [12]:
# !pip install dash_table